In [12]:
import pandas as pd
import numpy as np
import sys
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

In [40]:
cols = 4
col_names = []
for i in range(cols):
    col_names.append( 'r' + str(i+1))

rows = 10000
train_rows = int(rows * 0.75)

df = pd.DataFrame(np.random.randn(rows, cols), columns=col_names)
X = df
y = df.r1/df.r2  # + 0.0000000000000001) + df.r3 * df.r4

X_train = X[:train_rows]
y_train = y[:train_rows]
X_test = X[train_rows:]
y_test = y[train_rows:]


In [60]:
mode = 'regression'
params =  {
	'task': 'train',
	'boosting_type': 'gbdt',
	'objective': 'regression' if mode == 'regression' else 'binary',
	'metric': 'rmse',
	"learning_rate": 0.01,
	"num_leaves": 200,
	"feature_fraction": 0.70,
	"bagging_fraction": 0.70,
	'bagging_freq': 4,
	"max_depth": -1,
        "verbosity" : -1,
	"reg_alpha": 0.3,
	"reg_lambda": 0.1,
	#"min_split_gain":0.2,
	"min_child_weight":10,
	'zero_as_missing':True,
        'num_threads': 4,
    }



{'task': 'train',
 'boosting_type': 'gbdt',
 'objective': 'regression',
 'metric': 'rmse',
 'learning_rate': 0.01,
 'num_leaves': 200,
 'feature_fraction': 0.7,
 'bagging_fraction': 0.7,
 'bagging_freq': 4,
 'max_depth': -1,
 'verbosity': -1,
 'reg_alpha': 0.3,
 'reg_lambda': 0.1,
 'min_child_weight': 10,
 'zero_as_missing': True,
 'num_threads': 4}

In [61]:
prev_rmse = None
for d in range(2,16):
    #lgb_model = lgb.LGBMModel()
    est = lgb.LGBMRegressor(max_depth = d)
    est.fit(X_train, y_train)
    predict = est.predict( X_test)
    mse = mean_squared_error(y_test, predict)
    rmse = mse ** 0.5
    print(rmse)
    if (not prev_rmse is None) and prev_rmse < rmse:
        print('early finish at ', d)
        break
    prev_rmse = rmse

46.05779102175627
43.515665264595775
44.41246281524321
early finish at  4


In [62]:
est.get_params()
#params = {'key': [1,2,3] }
#param_name = list(params.keys())[0]
#param_values = params[param_name]
#print(param_values[0])

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': 4,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': True,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [37]:
df.head()

,r1,r2,r3,r4
0,0.164489,-0.904089,1.534742,0.698192
1,0.971173,0.606685,-0.977056,-1.847631
2,-1.783051,-1.537277,-0.680089,1.497183
3,-0.275111,-0.579068,2.271152,-0.302265
4,0.582036,-1.002653,-0.298985,-0.494816
